In [1]:
import sys
import neal
import greedy
import tabu
import time
import numpy as np

from pathlib import Path
current_path = Path().resolve()
sys.path.append(str(current_path / '../code/'))
from experiment import Experiment
from table import Table
from visualisation import Figure

# Load the data

In [2]:
data_folder = '../data/Travelling Salesman Problem/Large/'

file_names = ['berlin52.npz', 'brazil58.npz', 'st70.npz']

loaded_files = [np.load(data_folder + file) for file in file_names]

obj_qubos = [i['cost_function_qubo'] for i in loaded_files]
obj_constants = [i['cost_function_constant'] for i in loaded_files]
con_qubos = [i['constraint_function_qubo'] for i in loaded_files]
con_constants = [i['constraint_function_constant'] for i in loaded_files]

# Prepare the data

Here we already get the qubos and constraints. So we only need to calculate penalties and get the full QUBOs.

In [3]:
minimisation = True # This is a minimisation problem
QUBOs, penalties = Experiment.data_prep_light(obj_qubos, con_qubos, 'Verma&Lewis', minimisation)
qubo_sizes = [max(qubo, key=tuple)[0] + 1 for qubo in QUBOs]

# Run experiments

In [4]:
# The number of samples we want the sampler to return
repeats = 60

## Greedy

In [5]:
greedy_sampler = greedy.SteepestDescentSampler()
greedy_runs = Experiment.run_sampler(QUBOs, obj_qubos, obj_constants, con_qubos, con_constants, 
                                     greedy_sampler, repeats, num_reads=8)

100.0 %


## Simulated Annealing

In [6]:
sa_sampler = neal.SimulatedAnnealingSampler()
sa_runs = Experiment.run_sampler(QUBOs, obj_qubos, obj_constants, con_qubos, con_constants, 
                                 sa_sampler, repeats, num_reads=1)

100.0 %


## Tabu

In [7]:
tabu_sampler = tabu.TabuSampler()
tabu_runs = Experiment.run_sampler(QUBOs, obj_qubos, obj_constants, con_qubos, con_constants, 
                                   tabu_sampler, repeats, timeout=4000)

100.0 %


# Record the results

In [8]:
greedy_results = Table.record_results(greedy_runs, qubo_sizes, penalties, repeats, minimisation)
sa_results = Table.record_results(sa_runs, qubo_sizes, penalties, repeats, minimisation)
tabu_results = Table.record_results(tabu_runs, qubo_sizes, penalties, repeats, minimisation)

# Display the first repetition table
rep = 0
Table.display_side_by_side(greedy_results[rep], sa_results[rep], tabu_results[rep], titles=['Greedy', 'SA', 'Tabu'])

,Size,Penalty,Objective Function,Broken Constraints,Energy (minimisation)
0,2601,109810,15308,0,15308
1,3249,571102,50784,0,50784
2,4761,10055,1340,0,1340
,Size,Penalty,Objective Function,Broken Constraints,Energy (minimisation)
0,2601,109810,28645,0,28645
1,3249,571102,127112,0,127112
2,4761,10055,3577,0,3577
,Size,Penalty,Objective Function,Broken Constraints,Energy (minimisation)
0,2601,109810,11137,0,11137
1,3249,571102,43539,0,43539


# Explore the results

In [9]:
# Show total energies of all tries in all problems in a single df
energies_greedy = Table.columns_to_table(greedy_results, 'Energy (minimisation)')
energies_sa = Table.columns_to_table(sa_results, 'Energy (minimisation)')
energies_tabu = Table.columns_to_table(tabu_results, 'Energy (minimisation)')

energies_tabu

,Energy (minimisation) 0,Energy (minimisation) 1,Energy (minimisation) 2,Energy (minimisation) 3,Energy (minimisation) 4,Energy (minimisation) 5,Energy (minimisation) 6,Energy (minimisation) 7,Energy (minimisation) 8,Energy (minimisation) 9,...,Energy (minimisation) 50,Energy (minimisation) 51,Energy (minimisation) 52,Energy (minimisation) 53,Energy (minimisation) 54,Energy (minimisation) 55,Energy (minimisation) 56,Energy (minimisation) 57,Energy (minimisation) 58,Energy (minimisation) 59
0,11137,10735,10695,10637,10709,11118,10338,10590,11375,10670,...,10607,10337,10439,10316,10323,10466,11043,10444,10988,10762
1,43539,35983,41453,38467,43471,47929,41536,40090,46279,43932,...,42323,42433,40539,43226,46348,42503,44631,41999,40966,40143
2,1331,1391,1704,1255,1426,1132,1671,1332,1329,1341,...,1257,1230,1563,1202,1513,1139,1193,1317,1498,1395


In [10]:
# Show number of broken constraints of all tries in all problems in a single df
broken_constraints_greedy = Table.columns_to_table(greedy_results, 'Broken Constraints')
broken_constraints_sa = Table.columns_to_table(sa_results, 'Broken Constraints')
broken_constraints_tabu = Table.columns_to_table(tabu_results, 'Broken Constraints')

broken_constraints_greedy

,Broken Constraints 0,Broken Constraints 1,Broken Constraints 2,Broken Constraints 3,Broken Constraints 4,Broken Constraints 5,Broken Constraints 6,Broken Constraints 7,Broken Constraints 8,Broken Constraints 9,...,Broken Constraints 50,Broken Constraints 51,Broken Constraints 52,Broken Constraints 53,Broken Constraints 54,Broken Constraints 55,Broken Constraints 56,Broken Constraints 57,Broken Constraints 58,Broken Constraints 59
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# Show feasible solutions
feasible_full_greedy = Table.feasibility_table(greedy_results)
feasible_full_sa = Table.feasibility_table(sa_results)
feasible_full_tabu = Table.feasibility_table(tabu_results)

feasible_full_tabu

,Feasible 0,Feasible 1,Feasible 2,Feasible 3,Feasible 4,Feasible 5,Feasible 6,Feasible 7,Feasible 8,Feasible 9,...,Feasible 50,Feasible 51,Feasible 52,Feasible 53,Feasible 54,Feasible 55,Feasible 56,Feasible 57,Feasible 58,Feasible 59
0,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True


This is the final and the most important table. It displays the number of feasible solutions achieved for every problem, the feasibility rate (feasible_soultions/total_solutions), the mean energy of the solutions and the standard deviation.

In [12]:
# Calculate number of feasible solutions with mean and SD (in all runs)
feasible_greedy = Table.feasibility_statistic(greedy_results)
feasible_sa = Table.feasibility_statistic(sa_results)
feasible_tabu = Table.feasibility_statistic(tabu_results)

# Display the table
Table.display_side_by_side(feasible_greedy, feasible_sa, feasible_tabu, titles=['Greedy', 'SA', 'Tabu'])

,Feasible,Feasibility rate,Energy mean,Energy SD
0,60.0,1.0,14494.133333,608.637446
1,60.0,1.0,53272.516667,3406.836081
2,60.0,1.0,1460.800000,84.937425
Total,180.0,3.0,69227.450000,4100.410952
Mean,60.0,1.0,23075.816667,1366.803651
SD,0.0,0.0,26950.834173,1786.019223
,Feasible,Feasibility rate,Energy mean,Energy SD
0,60.0,1.0,29659.166667,1978.891293
1,60.0,1.0,123712.166667,8588.782174
2,60.0,1.0,3672.950000,178.831843


# Save results

In [13]:
data_folder = '../Data/Produced/Travelling Salesman Problem/Large/'
broken_constraints_greedy.to_pickle(data_folder + 'vlewis_tspL_greedy_broken_constraints.pkl')
broken_constraints_sa.to_pickle(data_folder + 'vlewis_tspL_sa_broken_constraints.pkl')
broken_constraints_tabu.to_pickle(data_folder + 'vlewis_tspL_tabu_broken_constraints.pkl')

feasible_greedy.to_pickle(data_folder + 'vlewis_tspL_greedy_feasible.pkl')
feasible_sa.to_pickle(data_folder + 'vlewis_tspL_sa_feasible.pkl')
feasible_tabu.to_pickle(data_folder + 'vlewis_tspL_tabu_feasible.pkl')